In [3]:
import pandas as pd

# Load datasets using raw strings to avoid issues with backslashes
movies = pd.read_csv(r'C:\Users\Anush\Downloads\movies (1).csv')  # Your movies dataset
ratings = pd.read_csv(r'C:\Users\Anush\Downloads\ratings.csv')  # Your ratings dataset

# Print the first few rows of each dataset
print(movies.head())
print(ratings.head())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [5]:
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')
print(user_movie_matrix.head())


movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     4.0     NaN     NaN     NaN     NaN     NaN   

movieId  9       10      ...  161084  161155  161594  161830  161918  161944  \
userId                   ...                                                   
1           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     4.0  ...     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     4.0  ...     NaN     NaN     NaN     NaN     NaN     N

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Fill NaN with 0 for the similarity calculation
user_movie_matrix_filled = user_movie_matrix.fillna(0)

# Compute similarity between users
user_similarity = cosine_similarity(user_movie_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

print(user_similarity_df.head())


userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.000000  0.000000  0.074482  0.016818  0.000000  0.083884   
2       0.000000  1.000000  0.124295  0.118821  0.103646  0.000000  0.212985   
3       0.000000  0.124295  1.000000  0.081640  0.151531  0.060691  0.154714   
4       0.074482  0.118821  0.081640  1.000000  0.130649  0.079648  0.319745   
5       0.016818  0.103646  0.151531  0.130649  1.000000  0.063796  0.095888   

userId       8         9         10   ...       662       663       664  \
userId                                ...                                 
1       0.000000  0.012843  0.000000  ...  0.000000  0.000000  0.014474   
2       0.113190  0.113333  0.043213  ...  0.477306  0.063202  0.077745   
3       0.249781  0.134475  0.114672  ...  0.161205  0.064198  0.176134   
4       0.191013  0.030417  0.137186  ...  0.114319  0.047228  0

In [8]:
def recommend_movies(user_id, user_movie_matrix, user_similarity_df, movies, n_recommendations=5):
    # Get similarity scores for the user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index
    similar_user_ratings = user_movie_matrix.loc[similar_users]
    
    # Compute weighted ratings
    weighted_ratings = similar_user_ratings.mul(user_similarity_df[user_id], axis=0)
    weighted_average_ratings = weighted_ratings.sum(axis=0) / user_similarity_df[user_id].sum()
    
    # Filter out movies the user has already rated
    already_rated = user_movie_matrix.loc[user_id].dropna().index
    recommendations = weighted_average_ratings.drop(already_rated).sort_values(ascending=False).head(n_recommendations)
    
    recommended_movies = movies.loc[movies['movieId'].isin(recommendations.index)]
    return recommended_movies

# Provide recommendations for a specific user
user_id = 1  # Change this to the user you want recommendations for
recommended_movies = recommend_movies(user_id, user_movie_matrix, user_similarity_df, movies)

print("Recommended Movies:")
print(recommended_movies)


Recommended Movies:
     movieId                                              title  \
232      260          Star Wars: Episode IV - A New Hope (1977)   
266      296                                Pulp Fiction (1994)   
284      318                   Shawshank Redemption, The (1994)   
953     1196  Star Wars: Episode V - The Empire Strikes Back...   
955     1198  Raiders of the Lost Ark (Indiana Jones and the...   

                          genres  
232      Action|Adventure|Sci-Fi  
266  Comedy|Crime|Drama|Thriller  
284                  Crime|Drama  
953      Action|Adventure|Sci-Fi  
955             Action|Adventure  


In [12]:
# Print the first few rows of the movies dataset
print("Movies Dataset:")
print(movies.head())

# Print the first few rows of the ratings dataset
print("Ratings Dataset:")
print(ratings.head())

# Print the first few rows of the user-movie matrix
print("User-Movie Matrix:")
print(user_movie_matrix.head())


Movies Dataset:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
Ratings Dataset:
   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205
User-Movie Matrix:
movieId  1       2       3       4       5       6       7       8       \
userId                                      

In [15]:
# Print recommendations for a specific user
user_id = 2  # Change this to the user you want recommendations for
recommended_movies = recommend_movies(user_id, user_movie_matrix, user_similarity_df, movies)

print("Recommended Movies for User", user_id)
print(recommended_movies)


Recommended Movies for User 2
     movieId                                      title  \
0          1                           Toy Story (1995)   
232      260  Star Wars: Episode IV - A New Hope (1977)   
284      318           Shawshank Redemption, The (1994)   
344      380                           True Lies (1994)   
527      595                Beauty and the Beast (1991)   

                                              genres  
0        Adventure|Animation|Children|Comedy|Fantasy  
232                          Action|Adventure|Sci-Fi  
284                                      Crime|Drama  
344         Action|Adventure|Comedy|Romance|Thriller  
527  Animation|Children|Fantasy|Musical|Romance|IMAX  


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
movies = pd.read_csv(r'C:\Users\Anush\Downloads\movies (1).csv')
ratings = pd.read_csv(r'C:\Users\Anush\Downloads\ratings.csv')

# Create User-Item Matrix
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Fill NaN with 0 for the similarity calculation
user_movie_matrix_filled = user_movie_matrix.fillna(0)

# Compute similarity between users
user_similarity = cosine_similarity(user_movie_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Function to recommend movies based on user ID
def recommend_movies(user_id, user_movie_matrix, user_similarity_df, movies, n_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index
    similar_user_ratings = user_movie_matrix.loc[similar_users]
    weighted_ratings = similar_user_ratings.mul(user_similarity_df[user_id], axis=0)
    weighted_average_ratings = weighted_ratings.sum(axis=0) / user_similarity_df[user_id].sum()
    already_rated = user_movie_matrix.loc[user_id].dropna().index
    recommendations = weighted_average_ratings.drop(already_rated).sort_values(ascending=False).head(n_recommendations)
    recommended_movies = movies.loc[movies['movieId'].isin(recommendations.index)]
    return recommended_movies

# Function to search movies based on name and user ID
def search_movie_recommendations(movie_name, user_id, user_movie_matrix, user_similarity_df, movies, n_recommendations=5):
    movie_matches = movies[movies['title'].str.contains(movie_name, case=False)]
    if movie_matches.empty:
        print(f"No movies found matching '{movie_name}'")
        return pd.DataFrame()  # Return an empty DataFrame if no matches found

    movie_id = movie_matches.iloc[0]['movieId']
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index
    similar_user_ratings = user_movie_matrix.loc[similar_users]
    weighted_ratings = similar_user_ratings.mul(user_similarity_df[user_id], axis=0)
    weighted_average_ratings = weighted_ratings.sum(axis=0) / user_similarity_df[user_id].sum()
    already_rated = user_movie_matrix.loc[user_id].dropna().index
    recommendations = weighted_average_ratings.drop(already_rated).sort_values(ascending=False).head(n_recommendations)
    recommended_movies = movies.loc[movies['movieId'].isin(recommendations.index)]
    return recommended_movies

# Main loop for CLI
while True:
    try:
        user_id = int(input("Enter your user ID (or 0 to exit): "))
        if user_id == 0:
            break
        if user_id not in user_movie_matrix.index:
            raise ValueError("User ID not found")
        movie_name = input("Enter a movie name to search (leave blank to skip): ").strip()
        if movie_name:
            recommended_movies = search_movie_recommendations(movie_name, user_id, user_movie_matrix, user_similarity_df, movies)
            if not recommended_movies.empty:
                print(f"\nRecommended Movies for User {user_id} based on '{movie_name}':")
                print(recommended_movies[['title']])
        else:
            recommended_movies = recommend_movies(user_id, user_movie_matrix, user_similarity_df, movies)
            print(f"\nRecommended Movies for User {user_id}:")
            print(recommended_movies[['title']])
    except ValueError as e:
        print(f"Invalid input: {e}. Please enter a valid user ID.")


In [16]:
from flask import Flask, request, render_template
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)

# Load datasets
movies = pd.read_csv(r'C:\Users\Anush\Downloads\movies (1).csv')
ratings = pd.read_csv(r'C:\Users\Anush\Downloads\ratings.csv')

# Create User-Item Matrix
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Fill NaN with 0 for the similarity calculation
user_movie_matrix_filled = user_movie_matrix.fillna(0)

# Compute similarity between users
user_similarity = cosine_similarity(user_movie_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Function to recommend movies based on user ID
def recommend_movies(user_id, user_movie_matrix, user_similarity_df, movies, n_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index
    similar_user_ratings = user_movie_matrix.loc[similar_users]
    weighted_ratings = similar_user_ratings.mul(user_similarity_df[user_id], axis=0)
    weighted_average_ratings = weighted_ratings.sum(axis=0) / user_similarity_df[user_id].sum()
    already_rated = user_movie_matrix.loc[user_id].dropna().index
    recommendations = weighted_average_ratings.drop(already_rated).sort_values(ascending=False).head(n_recommendations)
    recommended_movies = movies.loc[movies['movieId'].isin(recommendations.index)]
    return recommended_movies

# Function to search movies based on name and user ID
def search_movie_recommendations(movie_name, user_id, user_movie_matrix, user_similarity_df, movies, n_recommendations=5):
    movie_matches = movies[movies['title'].str.contains(movie_name, case=False)]
    if movie_matches.empty:
        return pd.DataFrame()  # Return an empty DataFrame if no matches found

    movie_id = movie_matches.iloc[0]['movieId']
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index
    similar_user_ratings = user_movie_matrix.loc[similar_users]
    weighted_ratings = similar_user_ratings.mul(user_similarity_df[user_id], axis=0)
    weighted_average_ratings = weighted_ratings.sum(axis=0) / user_similarity_df[user_id].sum()
    already_rated = user_movie_matrix.loc[user_id].dropna().index
    recommendations = weighted_average_ratings.drop(already_rated).sort_values(ascending=False).head(n_recommendations)
    recommended_movies = movies.loc[movies['movieId'].isin(recommendations.index)]
    return recommended_movies

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    user_id = int(request.form['user_id'])
    movie_name = request.form.get('movie_name')
    if movie_name:
        recommendations = search_movie_recommendations(movie_name, user_id, user_movie_matrix, user_similarity_df, movies)
        if recommendations.empty:
            return render_template('recommend.html', user_id=user_id, recommendations=[], error="No movies found matching the name.")
    else:
        recommendations = recommend_movies(user_id, user_movie_matrix, user_similarity_df, movies)
    
    recommendations_list = recommendations['title'].to_list()
    return render_template('recommend.html', user_id=user_id, recommendations=recommendations_list)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Anush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
